In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
REGION = "us-central1"

In [6]:
from typing import NamedTuple

from google.cloud import aiplatform
from kfp.dsl import pipeline
from kfp.dsl import component
from kfp import compiler

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=f"{PROJECT_ID}-bucket")

In [8]:
@component()
def concat(a: str, b: str) -> str:
    return a + b

@component
def reverse(a: str) -> NamedTuple("outputs", [("before", str), ("after", str)]):
    return a, a[::-1]

@pipeline(name="bootkon-example-pipeline")
def basic_pipeline(a: str='stres', b: str='sed'):
    concat_task = concat(a=a, b=b)
    reverse_task = reverse(a=concat_task.output)

In [9]:
compiler.Compiler().compile(pipeline_func=basic_pipeline, package_path="pipeline.yaml")

In [ ]:
job = aiplatform.PipelineJob(
    display_name="bootkon-example-pipeline",
    template_path="pipeline.yaml",
    pipeline_root=f"gs://{PROJECT_ID}-bucket/pipeline-root"
)

job.run(sync=True)  # Set sync=False for asynchronous execution

Creating PipelineJob
PipelineJob created. Resource name: projects/888342260584/locations/us-central1/pipelineJobs/bootkon-example-pipeline-20250130154138
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/888342260584/locations/us-central1/pipelineJobs/bootkon-example-pipeline-20250130154138')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bootkon-example-pipeline-20250130154138?project=888342260584
PipelineJob projects/888342260584/locations/us-central1/pipelineJobs/bootkon-example-pipeline-20250130154138 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/888342260584/locations/us-central1/pipelineJobs/bootkon-example-pipeline-20250130154138 current state:
PipelineState.PIPELINE_STATE_RUNNING


In [11]:
!mkdir -p src

In [12]:
%%writefile src/train.py
#!/usr/bin/env python
# Train a simple neural network classifier on Vertex AI

import os
from pprint import pprint
pprint(dict(os.environ))
import random
random.seed(1337)

import pandas as pd

from google.cloud import aiplatform, storage

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import metrics


BUCKET = os.environ["AIP_MODEL_DIR"].split("/")[2]

aiplatform.init(project=os.environ["CLOUD_ML_PROJECT_ID"],
                location=os.environ["CLOUD_ML_REGION"],
                staging_bucket=BUCKET)

# Load data
x_train = pd.read_csv(f"gs://{BUCKET}/vertex-data/x_train.csv")
y_train = pd.read_csv(f"gs://{BUCKET}/vertex-data/y_train.csv")["Class"]

# Train model
model = Sequential([
    Input(shape=(x_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),  # Hidden layer with 8 neurons
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=[metrics.AUC()])

history = model.fit(x_train.values, y_train.values, 
                    epochs=4, 
                    batch_size=32, 
                    validation_split=0.2, 
                    verbose=1)

# Upload model to Cloud Storage
model.save(os.environ["AIP_MODEL_DIR"])

Overwriting src/train.py


Let's run the training job! It will take around 6 minutes, but the provisioning of the job may take longer if there are a lot of people requesting resources at the moment.

While it is running, please go to the [`Training Jobs` in the Vertex AI Console](https://console.cloud.google.com/vertex-ai/training/training-pipelines) and click on the training job where you see **Status: Training**. The training job is based on a more general concept called `CustomJob` and adds functionality such as automatic model upload to Cloud Storage and registering the model to the model registry. Hence, to see details about the running job, click on the **Custom Job** and then **View Logs**.

***
<font color="red">While you wait for the next code chunk to complete (10-20min), you can already start going through the notebook named `bootkon_vertex_pipelines.ipynb`. Come back here regularly and continue when it has finished.</font>
***



We also deploy to model to an endpoint. Go to [`Online Predictions` in Vertex AI](https://console.cloud.google.com/vertex-ai/endpoints). Notice how the above command first creates an `Endpoint` and then deploys our `Model` to this endpoint.
